In [1]:
import numpy as np
import pandas as pd
from sklearn.op

### 1

In [2]:
data = np.loadtxt('../../../hw/14第14章  综合评价方法习题解答/ti14_1.txt')


array([[ 99.89, 103.69,  97.42, 101.11,  97.21],
       [ 96.91, 124.78,  66.44, 143.96,  88.36],
       [102.63, 101.85, 104.39, 100.94, 100.64],
       [ 98.47, 103.16, 109.17, 104.39,  91.9 ],
       [ 87.51,  90.27,  93.77,  94.33,  85.21],
       [108.35, 106.39, 142.35, 121.91, 158.61],
       [ 71.67, 137.16,  97.65, 171.31, 204.52],
       [103.25, 100.  , 100.  ,  99.13, 100.22],
       [171.2 ,  51.35,  15.9 ,  53.72,  20.78]])